In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *

import joblib
from datetime import timedelta
import tensorflow as tf
from sklearn.preprocessing import normalize

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
def create_window(dataset, win_size, start_index=0, end_index=None):
    data = []
    start_index = start_index + win_size
    if end_index is None:
        end_index = len(dataset)

    for i in range(start_index, end_index):
        indices = range(i-win_size, i)
        slices = dataset.iloc[indices]
        if slices.index.max() - slices.index.min() < timedelta(milliseconds=win_size * 300):
            data.append(slices[scanners].values)
    return np.array(data)

In [5]:
def impute(data):
    data.update(data.groupby(pd.Grouper(freq='100ms')).mean())
    data.update(data.rolling('5s').mean())
    data.ffill(inplace=True)

def linear_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models_2/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.reshape((-1, 1)))).repeat(data.shape[0])
        ss.index = data.index
        data[s].fillna(ss, inplace=True)

In [6]:
ble = read_ble('../Data/ble.csv')

In [7]:
coor = np.array([22, 6])

In [8]:
model = tf.keras.models.load_model("../Models/LSTM_Stationary_W10.h5")

In [9]:
client = InfluxDBClient()

In [10]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [11]:
data = format_data(data)

In [12]:
impute(data)
linear_impute(data)
data[scanners] = normalize(data[scanners] + 100)

In [13]:
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2020-01-22 02:07:58.124000+00:00,0.190701,0.322646,0.235476,0.237339,0.264054,0.302561,0.264384,0.165541,0.222076,0.108318,0.227840,0.311378,0.309593,0.114715,0.337713,0.150461,0.183221
2020-01-22 02:07:58.233000+00:00,0.193121,0.326739,0.238463,0.240350,0.267404,0.306400,0.267738,0.167641,0.224894,0.109693,0.230731,0.315328,0.313520,0.116170,0.341998,0.152369,0.094298
2020-01-22 02:07:58.297000+00:00,0.195211,0.330275,0.190637,0.242951,0.270298,0.309715,0.270635,0.169455,0.227327,0.110880,0.233228,0.318740,0.316913,0.117428,0.345699,0.154018,0.095319
2020-01-22 02:07:58.586000+00:00,0.193818,0.327918,0.189277,0.241217,0.268369,0.307505,0.268704,0.168246,0.225705,0.110088,0.231563,0.316466,0.336492,0.116590,0.343232,0.152919,0.094638
2020-01-22 02:07:58.658000+00:00,0.192412,0.325539,0.187904,0.239467,0.292295,0.305274,0.266755,0.167026,0.224068,0.109290,0.229884,0.314170,0.334051,0.115744,0.340742,0.151810,0.093952
2020-01-22 02:07:58.757000+00:00,0.193861,0.327992,0.189319,0.241271,0.294497,0.307574,0.268765,0.168284,0.189319,0.110113,0.231616,0.316537,0.336568,0.116616,0.343309,0.152954,0.094660
2020-01-22 02:07:58.799000+00:00,0.192076,0.324971,0.187576,0.239049,0.291785,0.333468,0.266289,0.166734,0.187576,0.109099,0.229483,0.313622,0.333468,0.115542,0.340147,0.151545,0.093788
2020-01-22 02:07:58.807000+00:00,0.193358,0.327141,0.188828,0.240645,0.293733,0.335695,0.268067,0.167847,0.188828,0.109827,0.231015,0.293733,0.335695,0.116313,0.342418,0.152557,0.094414
2020-01-22 02:07:58.838000+00:00,0.193358,0.327141,0.188828,0.240645,0.293733,0.335695,0.268067,0.167847,0.188828,0.109827,0.231015,0.293733,0.335695,0.116313,0.342418,0.152557,0.094414
2020-01-22 02:07:58.965000+00:00,0.198222,0.335369,0.193578,0.246698,0.301121,0.344138,0.274810,0.172069,0.193578,0.112590,0.236825,0.301121,0.344138,0.119239,0.268858,0.156394,0.096789


In [14]:
data = create_window(data, 10)

In [15]:
coor = model.predict(data).mean(axis=0).mean(axis=0)

In [16]:
np.expand_dims(coor, axis=0)

array([[22.087639 ,  7.0691085]], dtype=float32)